In [2]:
!pip install fastapi nest-asyncio pyngrok uvicorn mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00


In [3]:
!ngrok config add-authtoken 2wp8F8N7kxSiPpYoWHXkzenmEdt_738Ld4iuF3qVEeEYUfbzL

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import cv2
import torch
import numpy as np
import mysql.connector
import requests
from torchvision import transforms
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from torch.utils.data import Dataset, DataLoader
from fastapi import FastAPI, Path as FastAPIPath, BackgroundTasks
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import asyncio
import shutil
from typing import Dict, Optional, List
import threading
import time
from datetime import datetime
import traceback
from google.colab import drive
from pathlib import Path
import json
import pytz

# Mount Google Drive (sẽ yêu cầu xác thực)
drive.mount('/content/drive')

# FastAPI setup
app = FastAPI(title="API Huấn luyện mô hình nhận diện bạo lực")
nest_asyncio.apply()

# Thông tin kết nối MySQL
DB_CONFIG = {
    'host': 'sql12.freesqldatabase.com',
    'user': 'sql12779017',
    'password': 'qCN1zWvYiV',
    'database': 'sql12779017',
    'port': 3306
}

# Đường dẫn cơ sở - lưu trong Google Drive
DRIVE_BASE_PATH = "/content/drive/MyDrive/pttk"
BASE_PATH = os.path.join(DRIVE_BASE_PATH, "data")
MODEL_PATH = os.path.join(DRIVE_BASE_PATH, "models")

# Tạo thư mục trong Google Drive nếu chưa tồn tại
os.makedirs(DRIVE_BASE_PATH, exist_ok=True)
os.makedirs(BASE_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

# Cấu hình huấn luyện mặc định
TRAIN_CONFIG = {
    "epochs": 3,
    "batch_size": 2,
    "learning_rate": 1e-5,
    "num_frames": 8
}

# Theo dõi các tác vụ huấn luyện
training_tasks = {
    "active": None,  # ID của mô hình đang huấn luyện
    "queue": [],     # Hàng đợi các mô hình chờ huấn luyện
    "status": {}     # Trạng thái của từng mô hình
}

# Lock để đồng bộ hóa truy cập vào training_tasks
task_lock = threading.Lock()

# Hàm kết nối tới cơ sở dữ liệu
def get_db_connection():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        print(f"Lỗi kết nối đến cơ sở dữ liệu: {e}")
        return None

# Hàm cập nhật trạng thái mô hình trong database
def update_model_status_in_db(model_id: int, trang_thai: int, thoi_gian_bat_dau: datetime = None,
                              thoi_gian_ket_thuc: datetime = None, thong_so_huan_luyen: dict = None,
                              do_chinh_xac: float = None, duong_dan_mo_hinh: str = None):
    try:
        conn = get_db_connection()
        if not conn:
            print("Không thể kết nối CSDL để cập nhật trạng thái")
            return False

        cursor = conn.cursor()

        # Xây dựng câu UPDATE động dựa trên tham số
        updates = ["trang_thai = %s"]
        values = [trang_thai]

        if thoi_gian_bat_dau:
            updates.append("thoi_gian_bat_dau = %s")
            values.append(thoi_gian_bat_dau.strftime('%Y-%m-%d %H:%M:%S'))

        if thoi_gian_ket_thuc:
            updates.append("thoi_gian_ket_thuc = %s")
            values.append(thoi_gian_ket_thuc.strftime('%Y-%m-%d %H:%M:%S'))

        if thong_so_huan_luyen:
            updates.append("thong_so_huan_luyen = %s")
            values.append(json.dumps(thong_so_huan_luyen))

        if do_chinh_xac is not None:
            updates.append("do_chinh_xac = %s")
            values.append(do_chinh_xac)

        if duong_dan_mo_hinh:
            updates.append("duong_dan_mo_hinh = %s")
            values.append(duong_dan_mo_hinh)

        # Thêm điều kiện WHERE
        update_query = f"""
        UPDATE mo_hinh_da_huan_luyen
        SET {', '.join(updates)}
        WHERE id = %s
        """
        values.append(model_id)

        cursor.execute(update_query, values)
        conn.commit()

        print(f"✅ Đã cập nhật trạng thái mô hình {model_id}: {trang_thai}")
        cursor.close()
        conn.close()
        return True

    except Exception as e:
        print(f"Lỗi cập nhật trạng thái CSDL: {e}")
        print(traceback.format_exc())
        return False

# Hàm trích xuất frames từ video
def extract_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        print(f"⚠ Video {video_path} quá ngắn!")
        return None

    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Chuyển về RGB
        frames.append(frame)

    cap.release()
    return frames if len(frames) == num_frames else None

# Dataset đọc trực tiếp video
class ActionVideoDataset(Dataset):
    def __init__(self, data_folder, processor, num_frames=8):
        self.data_folder = data_folder
        self.processor = processor
        self.num_frames = num_frames
        self.video_paths = []
        self.targets = []
        self.labels = {}

        # Lấy nhãn từ cấu trúc thư mục
        label_id = 0
        for action in os.listdir(data_folder):
            action_path = os.path.join(data_folder, action)
            if os.path.isdir(action_path):
                self.labels[action] = label_id
                label_id += 1

                # Lấy tất cả video trong thư mục nhãn
                for video in os.listdir(action_path):
                    if video.endswith(('.mp4', '.avi')):
                        video_path = os.path.join(action_path, video)
                        self.video_paths.append(video_path)
                        self.targets.append(self.labels[action])

        print(f"Đã tìm thấy {len(self.labels)} nhãn: {self.labels}")
        print(f"Tổng số video: {len(self.video_paths)}")

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.targets[idx]

        frames = extract_frames(video_path, num_frames=self.num_frames)
        if frames is None:
            return None

        inputs = self.processor(images=frames, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0), torch.tensor(label)

# Hàm tải video từ cơ sở dữ liệu
def download_videos_for_model(model_id: int):
    try:
        conn = get_db_connection()
        if not conn:
            return False, "Không thể kết nối đến cơ sở dữ liệu"

        cursor = conn.cursor(dictionary=True)

        # Tạo thư mục chính nếu chưa tồn tại
        model_data_path = os.path.join(BASE_PATH, f"model_{model_id}")
        if os.path.exists(model_data_path):
            print(f"Xóa thư mục dữ liệu cũ: {model_data_path}")
            shutil.rmtree(model_data_path)  # Xóa thư mục cũ nếu tồn tại
        os.makedirs(model_data_path, exist_ok=True)

        # 1. Lấy thông tin về mô hình
        cursor.execute("SELECT * FROM mo_hinh_da_huan_luyen WHERE id = %s", (model_id,))
        model = cursor.fetchone()
        if not model:
            cursor.close()
            conn.close()
            return False, f"Không tìm thấy mô hình có ID = {model_id}"

        tap_du_lieu_id = model['tap_du_lieu_id']
        print(f"Lấy dữ liệu từ tập dữ liệu ID = {tap_du_lieu_id}")

        # 2. Lấy các mẫu bạo lực thuộc tập dữ liệu đó
        query = """
        SELECT mbl.*
        FROM tap_du_lieu_mau tdm
        JOIN mau_bao_luc mbl ON tdm.mau_bao_luc_id = mbl.id
        WHERE tdm.tap_du_lieu_id = %s
        """
        cursor.execute(query, (tap_du_lieu_id,))
        videos = cursor.fetchall()

        if not videos:
            cursor.close()
            conn.close()
            return False, f"Không có video nào trong tập dữ liệu ID = {tap_du_lieu_id}"

        print(f"Tìm thấy {len(videos)} video trong tập dữ liệu")

        # 3. Tạo thư mục và tải video
        video_count = 0
        for video in videos:
            duong_dan = video['duong_dan_video']
            nhan = video['nhan'] or "unknown"
            ten = video['ten'] or f"video_{video['id']}"

            folder_path = os.path.join(model_data_path, nhan)
            os.makedirs(folder_path, exist_ok=True)

            try:
                response = requests.get(duong_dan, stream=True, timeout=30)
                if response.status_code == 200:
                    video_path = os.path.join(folder_path, f"{ten}.mp4")
                    with open(video_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    print(f"✅ Lưu video {ten} vào {folder_path}")
                    video_count += 1
                else:
                    print(f"❌ Không thể tải {duong_dan} (status {response.status_code})")

                # Cập nhật trạng thái
                with task_lock:
                    if model_id in training_tasks["status"]:
                        training_tasks["status"][model_id]["progress"] = 10 + (video_count / len(videos) * 20)
                        training_tasks["status"][model_id]["message"] = f"Đã tải {video_count}/{len(videos)} video"

            except Exception as e:
                print(f"⚠️ Lỗi kết nối đến {duong_dan}: {e}")

        cursor.close()
        conn.close()

        if video_count == 0:
            return False, "Không tải được video nào"

        print(f"Đã tải xong {video_count}/{len(videos)} video vào {model_data_path}")
        return True, model_data_path

    except Exception as e:
        print(f"Lỗi trong quá trình tải video: {e}")
        print(traceback.format_exc())
        return False, f"Lỗi: {str(e)}"

# Hàm huấn luyện mô hình (chạy trong thread riêng)
def train_model_task(model_id: int):
    try:
        # Thời gian bắt đầu
        start_time = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh'))

        # Cập nhật trạng thái bắt đầu huấn luyện (2) và thời gian bắt đầu
        update_model_status_in_db(model_id, 2, datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')))

        # Cập nhật trạng thái
        with task_lock:
            training_tasks["active"] = model_id
            training_tasks["status"][model_id] = {
                "start_time": start_time.isoformat(),
                "progress": 0,
                "message": "Khởi động quá trình huấn luyện...",
                "completed": False,
                "success": False
            }

        # Cập nhật trạng thái
        def update_status(progress, message):
            print(f"[Mô hình {model_id}] {message} (Tiến độ: {progress}%)")
            with task_lock:
                if model_id in training_tasks["status"]:
                    training_tasks["status"][model_id]["progress"] = progress
                    training_tasks["status"][model_id]["message"] = message

        update_status(5, "Đang tải dữ liệu video...")

        # Tải video từ cơ sở dữ liệu
        success, result = download_videos_for_model(model_id)
        if not success:
            update_status(0, f"Lỗi tải dữ liệu: {result}")
            with task_lock:
                training_tasks["active"] = None
                training_tasks["status"][model_id]["completed"] = True
                training_tasks["status"][model_id]["success"] = False
            process_next_in_queue()
            return False

        data_path = result
        update_status(30, f"Đã tải xong dữ liệu vào {data_path}, đang khởi tạo mô hình...")

        # Kiểm tra xem có thư mục nhãn nào không
        label_dirs = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
        if not label_dirs:
            update_status(0, "Không tìm thấy thư mục nhãn trong dữ liệu")
            with task_lock:
                training_tasks["active"] = None
                training_tasks["status"][model_id]["completed"] = True
                training_tasks["status"][model_id]["success"] = False
            process_next_in_queue()
            return False

        print(f"Thư mục nhãn: {label_dirs}")

        # Tạo đối tượng labels từ thư mục
        labels_dict = {label: idx for idx, label in enumerate(label_dirs)}
        num_labels = len(labels_dict)
        print(f"Labels: {labels_dict}")

        # Khởi tạo mô hình và processor
        model_name = "facebook/timesformer-base-finetuned-k400"
        print(f"Tải processor từ {model_name}")
        processor = AutoImageProcessor.from_pretrained(model_name)

        print(f"Tải mô hình từ {model_name} với {num_labels} nhãn")
        model = TimesformerForVideoClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            ignore_mismatched_sizes=True
        )

        update_status(40, "Đang chuẩn bị dữ liệu huấn luyện...")

        # Chuẩn bị dữ liệu
        print(f"Tạo dataset từ {data_path}")
        train_dataset = ActionVideoDataset(data_path, processor, num_frames=TRAIN_CONFIG["num_frames"])

        if len(train_dataset) == 0:
            update_status(0, "Không có dữ liệu huấn luyện hợp lệ")
            with task_lock:
                training_tasks["active"] = None
                training_tasks["status"][model_id]["completed"] = True
                training_tasks["status"][model_id]["success"] = False
            process_next_in_queue()
            return False

        # Lọc các mẫu hợp lệ (không None)
        valid_data = []
        for i in range(len(train_dataset)):
            item = train_dataset[i]
            if item is not None:
                valid_data.append(item)

            # Cập nhật tiến trình
            if i % 5 == 0:
                update_status(40 + int((i / len(train_dataset)) * 10),
                              f"Đang xử lý dữ liệu: {i+1}/{len(train_dataset)}")

        if not valid_data:
            update_status(0, "Không tìm thấy mẫu video hợp lệ nào")
            with task_lock:
                training_tasks["active"] = None
                training_tasks["status"][model_id]["completed"] = True
                training_tasks["status"][model_id]["success"] = False
            process_next_in_queue()
            return False

        # Tạo DataLoader
        train_loader = DataLoader(valid_data, batch_size=TRAIN_CONFIG["batch_size"], shuffle=True)

        update_status(50, f"Bắt đầu huấn luyện với {len(valid_data)} mẫu video...")

        # Huấn luyện mô hình
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Sử dụng thiết bị: {device}")
        model.to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=TRAIN_CONFIG["learning_rate"])
        loss_fn = torch.nn.CrossEntropyLoss()

        avg_loss = 0
        for epoch in range(TRAIN_CONFIG["epochs"]):
            model.train()
            total_loss = 0
            batch_count = 0

            for batch_idx, batch in enumerate(train_loader):
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(pixel_values=inputs)
                loss = loss_fn(outputs.logits, targets)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                batch_count += 1

                # Cập nhật tiến trình
                progress_per_epoch = 40 / TRAIN_CONFIG["epochs"]
                current_epoch_progress = (batch_idx / len(train_loader)) * progress_per_epoch
                update_status(50 + int((epoch * progress_per_epoch) + current_epoch_progress),
                              f"Epoch {epoch+1}/{TRAIN_CONFIG['epochs']} - Batch {batch_idx+1}/{len(train_loader)}")

            avg_loss = total_loss / batch_count if batch_count > 0 else 0
            print(f"Epoch {epoch+1}/{TRAIN_CONFIG['epochs']} - Loss: {avg_loss:.4f}")
            update_status(50 + ((epoch+1) / TRAIN_CONFIG["epochs"]) * 40,
                          f"Epoch {epoch+1}/{TRAIN_CONFIG['epochs']} - Loss: {avg_loss:.4f}")

        # Lưu mô hình vào Google Drive
        try:
            model_save_path = os.path.join(MODEL_PATH, f"model_{model_id}")
            print(f"Bắt đầu lưu mô hình vào {model_save_path}")

            if os.path.exists(model_save_path):
                print(f"Xóa thư mục mô hình cũ: {model_save_path}")
                shutil.rmtree(model_save_path)

            os.makedirs(model_save_path, exist_ok=True)

            update_status(90, f"Đang lưu mô hình vào Drive tại {model_save_path}...")

            # Lưu mô hình
            print(f"Lưu mô hình...")
            model.save_pretrained(model_save_path)
            print(f"Lưu mô hình thành công")

            # Lưu processor
            print(f"Lưu processor...")
            processor.save_pretrained(model_save_path)
            print(f"Lưu processor thành công")

            # Lưu thông tin nhãn
            labels_file = os.path.join(model_save_path, "labels.json")
            print(f"Lưu nhãn vào {labels_file}")
            with open(labels_file, "w") as f:
                json.dump(labels_dict, f)

            # Kiểm tra files đã lưu
            saved_files = os.listdir(model_save_path)
            print(f"Files đã lưu trong {model_save_path}: {saved_files}")

            update_status(95, "Đang cập nhật thông tin mô hình vào cơ sở dữ liệu...")
        except Exception as e:
            print(f"Lỗi khi lưu mô hình: {e}")
            print(traceback.format_exc())

        # Chuẩn bị thông tin cập nhật
        accuracy = float(1.0 - avg_loss) if avg_loss <= 1.0 else 0.1
        absolute_model_path = os.path.abspath(model_save_path)

        # Chuẩn bị thông số huấn luyện dạng JSON
        thong_so_huan_luyen = {
            "epochs": TRAIN_CONFIG["epochs"],
            "batch_size": TRAIN_CONFIG["batch_size"],
            "learning_rate": TRAIN_CONFIG["learning_rate"],
            "num_frames": TRAIN_CONFIG["num_frames"],
            "num_labels": num_labels,
            "labels": labels_dict,
            "model_name": model_name,
            "device": device,
            "final_loss": avg_loss,
            "num_training_samples": len(valid_data)
        }

        end_time = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh'))

        # Cập nhật tất cả thông tin vào database với trạng thái hoàn thành (3)
        update_model_status_in_db(
            model_id=model_id,
            trang_thai=3,
            thoi_gian_bat_dau=start_time,
            thoi_gian_ket_thuc=end_time,
            thong_so_huan_luyen=thong_so_huan_luyen,
            do_chinh_xac=accuracy,
            duong_dan_mo_hinh=absolute_model_path
        )

        update_status(100, f"Huấn luyện hoàn tất! Mô hình được lưu tại {model_save_path}")

        # Đánh dấu hoàn thành
        with task_lock:
            training_tasks["active"] = None
            training_tasks["status"][model_id]["completed"] = True
            training_tasks["status"][model_id]["success"] = True
            training_tasks["status"][model_id]["end_time"] = end_time.isoformat()

        # Xử lý mô hình tiếp theo trong hàng đợi
        process_next_in_queue()

        return True

    except Exception as e:
        print(f"Lỗi trong quá trình huấn luyện: {e}")
        print(traceback.format_exc())

        # Cập nhật trạng thái lỗi
        update_model_status_in_db(model_id, 3, thoi_gian_ket_thuc=datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')))

        with task_lock:
            if model_id in training_tasks["status"]:
                training_tasks["status"][model_id]["message"] = f"Lỗi huấn luyện: {str(e)}"
                training_tasks["status"][model_id]["completed"] = True
                training_tasks["status"][model_id]["success"] = False
                training_tasks["status"][model_id]["end_time"] = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).isoformat()
            training_tasks["active"] = None

        # Xử lý mô hình tiếp theo trong hàng đợi
        process_next_in_queue()

        return False

# Xử lý mô hình tiếp theo trong hàng đợi
def process_next_in_queue():
    with task_lock:
        if not training_tasks["queue"] or training_tasks["active"] is not None:
            return

        # Lấy mô hình tiếp theo từ hàng đợi
        next_model_id = training_tasks["queue"].pop(0)
        print(f"Tiếp tục xử lý mô hình tiếp theo trong hàng đợi: {next_model_id}")

        # Khởi động một thread mới để huấn luyện
        # update_model_status_in_db(next_model_id, 2, datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')))
        thread = threading.Thread(target=train_model_task, args=(next_model_id,))
        thread.daemon = True
        thread.start()

# API endpoint thêm mô hình vào hàng đợi huấn luyện
@app.get("/train/{model_id}")
async def trigger_training(model_id: int = FastAPIPath(..., description="ID của mô hình cần huấn luyện")):
    # Kiểm tra ID mô hình trong cơ sở dữ liệu
    conn = get_db_connection()
    if not conn:
        return {"success": False, "message": "Không thể kết nối đến cơ sở dữ liệu"}

    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT id FROM mo_hinh_da_huan_luyen WHERE id = %s", (model_id,))
    model = cursor.fetchone()
    cursor.close()
    conn.close()

    if not model:
        return {"success": False, "message": f"Không tìm thấy mô hình có ID = {model_id}"}

    # Kiểm tra xem mô hình đã trong hàng đợi hoặc đang huấn luyện chưa
    with task_lock:
        if model_id == training_tasks["active"]:
            return {
                "success": False,
                "message": f"Mô hình ID: {model_id} đang được huấn luyện",
                "status": training_tasks["status"].get(model_id, {})
            }

        if model_id in training_tasks["queue"]:
            position = training_tasks["queue"].index(model_id) + 1
            return {
                "success": False,
                "message": f"Mô hình ID: {model_id} đã trong hàng đợi ở vị trí {position}",
                "queue_position": position
            }

        # Thêm vào hàng đợi
        if model_id in training_tasks["status"] and not training_tasks["status"][model_id]["completed"]:
            # Mô hình đang được xử lý
            return {
                "success": False,
                "message": f"Mô hình ID: {model_id} đang trong quá trình xử lý",
                "status": training_tasks["status"].get(model_id, {})
            }

        # Khởi tạo trạng thái
        training_tasks["status"][model_id] = {
            "queued_time": datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).isoformat(),
            "progress": 0,
            "message": "Đang chờ trong hàng đợi",
            "completed": False,
            "success": False
        }

        # Cập nhật trạng thái queue (1) vào database
        update_model_status_in_db(model_id, 1)

        # Nếu không có mô hình nào đang huấn luyện, bắt đầu huấn luyện ngay
        if training_tasks["active"] is None:
            # Khởi động thread mới để huấn luyện
            # update_model_status_in_db(model_id, 2, datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')))
            thread = threading.Thread(target=train_model_task, args=(model_id,))
            thread.daemon = True
            thread.start()
            return {
                "success": True,
                "message": f"Bắt đầu huấn luyện mô hình ID: {model_id}",
                "queue_position": 0
            }
        else:
            # Thêm vào hàng đợi
            training_tasks["queue"].append(model_id)
            queue_position = len(training_tasks["queue"])
            return {
                "success": True,
                "message": f"Đã thêm mô hình ID: {model_id} vào hàng đợi huấn luyện",
                "queue_position": queue_position
            }

# API endpoint kiểm tra trạng thái huấn luyện
@app.get("/status")
async def get_status():
    with task_lock:
        active_model = training_tasks["active"]
        active_status = training_tasks["status"].get(active_model, {}) if active_model else None
        queue_size = len(training_tasks["queue"])
        queue_models = training_tasks["queue"].copy()

        # Lấy trạng thái của tất cả các mô hình
        all_models_status = {model_id: status for model_id, status in training_tasks["status"].items()}

        return {
            "active_model": active_model,
            "active_status": active_status,
            "queue_size": queue_size,
            "queue_models": queue_models,
            "all_models_status": all_models_status
        }

# API endpoint kiểm tra trạng thái một mô hình cụ thể
@app.get("/status/{model_id}")
async def get_model_status(model_id: int = FastAPIPath(..., description="ID của mô hình cần kiểm tra")):
    with task_lock:
        if model_id not in training_tasks["status"]:
            return {"success": False, "message": f"Không tìm thấy trạng thái cho mô hình ID: {model_id}"}

        status = training_tasks["status"][model_id]
        is_active = (training_tasks["active"] == model_id)

        if not is_active and not status["completed"]:
            # Mô hình đang trong hàng đợi
            queue_position = training_tasks["queue"].index(model_id) + 1 if model_id in training_tasks["queue"] else 0
            return {
                "success": True,
                "is_active": False,
                "is_queued": True,
                "queue_position": queue_position,
                "status": status
            }

        return {
            "success": True,
            "is_active": is_active,
            "is_queued": False,
            "status": status
        }

# Khởi tạo ứng dụng
@app.on_event("startup")
async def startup_event():
    # Tạo thư mục lưu trữ nếu chưa tồn tại
    os.makedirs(BASE_PATH, exist_ok=True)
    os.makedirs(MODEL_PATH, exist_ok=True)
    print(f"Đã tạo thư mục: {BASE_PATH} và {MODEL_PATH}")

# API endpoint xóa trạng thái của một mô hình (để làm sạch)
@app.delete("/status/{model_id}")
async def delete_model_status(model_id: int = FastAPIPath(..., description="ID của mô hình cần xóa trạng thái")):
    with task_lock:
        if model_id not in training_tasks["status"]:
            return {"success": False, "message": f"Không tìm thấy trạng thái cho mô hình ID: {model_id}"}

        if model_id == training_tasks["active"]:
            return {"success": False, "message": f"Không thể xóa trạng thái của mô hình đang huấn luyện"}

        if model_id in training_tasks["queue"]:
            training_tasks["queue"].remove(model_id)

        del training_tasks["status"][model_id]

        return {"success": True, "message": f"Đã xóa trạng thái của mô hình ID: {model_id}"}

# Chạy ứng dụng
if __name__ == "__main__":
    try:
        # Kiểm tra kết nối Drive
        print(f"Thư mục Drive đã kết nối tại: {DRIVE_BASE_PATH}")
        print(f"Nội dung thư mục DRIVE_BASE_PATH: {os.listdir(DRIVE_BASE_PATH) if os.path.exists(DRIVE_BASE_PATH) else 'Không tồn tại'}")
        print(f"Nội dung thư mục BASE_PATH: {os.listdir(BASE_PATH) if os.path.exists(BASE_PATH) else 'Không tồn tại'}")
        print(f"Nội dung thư mục MODEL_PATH: {os.listdir(MODEL_PATH) if os.path.exists(MODEL_PATH) else 'Không tồn tại'}")

        # Khởi động ngrok
        public_url = ngrok.connect(8000)
        print(f"🌐 Public URL: {public_url}")

        # Chạy FastAPI
        uvicorn.run(app, host="0.0.0.0", port=8000)
    except Exception as e:
        print(f"Lỗi khởi động ứng dụng: {e}")
        print(traceback.format_exc())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Thư mục Drive đã kết nối tại: /content/drive/MyDrive/pttk
Nội dung thư mục DRIVE_BASE_PATH: ['data', 'models']
Nội dung thư mục BASE_PATH: ['model_3', 'model_4']
Nội dung thư mục MODEL_PATH: ['model_3']


<ipython-input-2-b32f5cf6e88e>:687: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


🌐 Public URL: NgrokTunnel: "https://fb8d-34-13-155-234.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [2961]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Đã tạo thư mục: /content/drive/MyDrive/pttk/data và /content/drive/MyDrive/pttk/models
✅ Đã cập nhật trạng thái mô hình 8: 1
INFO:     14.0.22.145:0 - "GET /train/8 HTTP/1.1" 200 OK
✅ Đã cập nhật trạng thái mô hình 8: 2
[Mô hình 8] Đang tải dữ liệu video... (Tiến độ: 5%)
Lấy dữ liệu từ tập dữ liệu ID = 1
Tìm thấy 2 video trong tập dữ liệu
✅ Lưu video da vào /content/drive/MyDrive/pttk/data/model_8/da
✅ Lưu video dam vào /content/drive/MyDrive/pttk/data/model_8/dam
Đã tải xong 2/2 video vào /content/drive/MyDrive/pttk/data/model_8
[Mô hình 8] Đã tải xong dữ liệu vào /content/drive/MyDrive/pttk/data/model_8, đang khởi tạo mô hình... (Tiến độ: 30%)
Thư mục nhãn: ['da', 'dam']
Labels: {'da': 0, 'dam': 1}
Tải processor từ facebook/timesformer-base-finetuned-k400


...

[Mô hình 8] Đang chuẩn bị dữ liệu huấn luyện... (Tiến độ: 40%)
Tạo dataset từ /content/drive/MyDrive/pttk/data/model_8
Đã tìm thấy 2 nhãn: {'da': 0, 'dam': 1}
Tổng số video: 2
[Mô hình 8] Đang xử lý dữ liệu: 1/2 (Tiến độ: 40%)
[Mô hình 8] Bắt đầu huấn luyện với 2 mẫu video... (Tiến độ: 50%)
Sử dụng thiết bị: cpu
[Mô hình 8] Epoch 1/3 - Batch 1/1 (Tiến độ: 50%)
Epoch 1/3 - Loss: 0.8207
[Mô hình 8] Epoch 1/3 - Loss: 0.8207 (Tiến độ: 63.33333333333333%)
[Mô hình 8] Epoch 2/3 - Batch 1/1 (Tiến độ: 63%)
Epoch 2/3 - Loss: 0.4206
[Mô hình 8] Epoch 2/3 - Loss: 0.4206 (Tiến độ: 76.66666666666666%)
[Mô hình 8] Epoch 3/3 - Batch 1/1 (Tiến độ: 76%)
Epoch 3/3 - Loss: 0.2033
[Mô hình 8] Epoch 3/3 - Loss: 0.2033 (Tiến độ: 90.0%)
Bắt đầu lưu mô hình vào /content/drive/MyDrive/pttk/models/model_8
[Mô hình 8] Đang lưu mô hình vào Drive tại /content/drive/MyDrive/pttk/models/model_8... (Tiến độ: 90%)
Lưu mô hình...
Lưu mô hình thành công
Lưu processor...
Lưu processor thành công
Lưu nhãn vào /content/dri

Some weights of TimesformerForVideoClassification were not initialized from the model checkpoint at facebook/timesformer-base-finetuned-k400 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Mô hình 7] Đang chuẩn bị dữ liệu huấn luyện... (Tiến độ: 40%)
Tạo dataset từ /content/drive/MyDrive/pttk/data/model_7
Đã tìm thấy 2 nhãn: {'da': 0, 'dam': 1}
Tổng số video: 3
[Mô hình 7] Đang xử lý dữ liệu: 1/3 (Tiến độ: 40%)
[Mô hình 7] Bắt đầu huấn luyện với 3 mẫu video... (Tiến độ: 50%)
Sử dụng thiết bị: cpu
[Mô hình 7] Epoch 1/3 - Batch 1/2 (Tiến độ: 50%)
[Mô hình 7] Epoch 1/3 - Batch 2/2 (Tiến độ: 56%)
Epoch 1/3 - Loss: 1.0362
[Mô hình 7] Epoch 1/3 - Loss: 1.0362 (Tiến độ: 63.33333333333333%)
[Mô hình 7] Epoch 2/3 - Batch 1/2 (Tiến độ: 63%)
[Mô hình 7] Epoch 2/3 - Batch 2/2 (Tiến độ: 70%)
Epoch 2/3 - Loss: 0.3861
[Mô hình 7] Epoch 2/3 - Loss: 0.3861 (Tiến độ: 76.66666666666666%)
[Mô hình 7] Epoch 3/3 - Batch 1/2 (Tiến độ: 76%)
[Mô hình 7] Epoch 3/3 - Batch 2/2 (Tiến độ: 83%)
Epoch 3/3 - Loss: 0.1649
[Mô hình 7] Epoch 3/3 - Loss: 0.1649 (Tiến độ: 90.0%)
Bắt đầu lưu mô hình vào /content/drive/MyDrive/pttk/models/model_7
[Mô hình 7] Đang lưu mô hình vào Drive tại /content/drive/MyDr

Some weights of TimesformerForVideoClassification were not initialized from the model checkpoint at facebook/timesformer-base-finetuned-k400 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Mô hình 9] Đang chuẩn bị dữ liệu huấn luyện... (Tiến độ: 40%)
Tạo dataset từ /content/drive/MyDrive/pttk/data/model_9
Đã tìm thấy 1 nhãn: {'da': 0}
Tổng số video: 1
[Mô hình 9] Đang xử lý dữ liệu: 1/1 (Tiến độ: 40%)
[Mô hình 9] Bắt đầu huấn luyện với 1 mẫu video... (Tiến độ: 50%)
Sử dụng thiết bị: cpu
[Mô hình 9] Epoch 1/3 - Batch 1/1 (Tiến độ: 50%)
Epoch 1/3 - Loss: 0.0000
[Mô hình 9] Epoch 1/3 - Loss: 0.0000 (Tiến độ: 63.33333333333333%)
[Mô hình 9] Epoch 2/3 - Batch 1/1 (Tiến độ: 63%)
Epoch 2/3 - Loss: 0.0000
[Mô hình 9] Epoch 2/3 - Loss: 0.0000 (Tiến độ: 76.66666666666666%)
[Mô hình 9] Epoch 3/3 - Batch 1/1 (Tiến độ: 76%)
Epoch 3/3 - Loss: 0.0000
[Mô hình 9] Epoch 3/3 - Loss: 0.0000 (Tiến độ: 90.0%)
Bắt đầu lưu mô hình vào /content/drive/MyDrive/pttk/models/model_9
[Mô hình 9] Đang lưu mô hình vào Drive tại /content/drive/MyDrive/pttk/models/model_9... (Tiến độ: 90%)
Lưu mô hình...
Lưu mô hình thành công
Lưu processor...
Lưu processor thành công
Lưu nhãn vào /content/drive/MyDrive